# Footprint Plots (Combined)

In [ ]:
from __future__ import annotations

from multiprocessing import Pool
from pathlib import Path

import matplotlib.patches as mpatches
import pandas as pd
from omegaconf import OmegaConf

config_folder = Path("../carps/configs/task")
paths = list(config_folder.glob("**/*.yaml"))
paths = [p for p in paths if "DUMMY" not in str(p)]


def read_task(p) -> dict:
    cfg = OmegaConf.load(p)
    task = OmegaConf.to_container(cfg.task)
    task["benchmark_id"] = cfg.benchmark_id
    task["task_id"] = cfg.task_id
    return task


with Pool() as pool:
    tasks = pool.map(read_task, paths)
tasks = pd.DataFrame(tasks)

In [ ]:
data = []
benchmark_id_list = tasks.benchmark_id.unique()
benchmark_id_list = [b for b in benchmark_id_list if ("Vizier" not in b) and (b != "HPOB")]
for benchmark_id in benchmark_id_list:
    tasks_id = tasks[tasks["benchmark_id"] == benchmark_id]
    BB = len(tasks_id[(tasks_id["n_objectives"] == 1) & (tasks_id["is_multifidelity"] is False)])
    MF = len(tasks_id[(tasks_id["n_objectives"] == 1) & (tasks_id["is_multifidelity"] is True)])
    MO = len(tasks_id[(tasks_id["n_objectives"] > 1) & (tasks_id["is_multifidelity"] is False)])
    MOMF = len(tasks_id[(tasks_id["n_objectives"] > 1) & (tasks_id["is_multifidelity"] is True)])
    dimensions = list(tasks_id["dimensions"])
    real = len(tasks_id[tasks_id["objective_function_approximation"] == "real"])
    tab = len(tasks_id[tasks_id["objective_function_approximation"] == "tabular"])
    surr = len(tasks_id[tasks_id["objective_function_approximation"] == "surrogate"])
    float = tasks_id["search_space_n_floats"].sum()
    int = tasks_id["search_space_n_integers"].sum()
    cat = tasks_id["search_space_n_categoricals"].sum()
    ord = tasks_id["search_space_n_ordinals"].sum()

    data.append(
        {
            "Scenario": {
                "BB": BB,
                "MF": MF,
                "MO": MO,
                "MOMF": MOMF,
            },
            "Dimensions": dimensions,
            "Objective Function": {"real": real, "tab": tab, "surr": surr},
            "HP Types": {
                "float": float,
                "int": int,
                "cat": cat,
                "ord": ord,
            },
        }
    )

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from carps.analysis.utils import savefig, setup_seaborn

setup_seaborn()

colors = dict(zip(benchmark_id_list, ["#88CCEE", "#117733", "#999933", "#DDCC77", "#CC6677", "#882255"], strict=False))

fig, axs = plt.subplots(1, 4, figsize=(20, 5))

scenarios = {}
obj_fun = {}
hp_types = {}
dimensions = {}

shift = 0.08
bar_width = 0.4

# Iterate over the dictionaries in the list
for i, data_entry in enumerate(data):
    # Iterate over the keys in the dictionary
    for j, (key, values) in enumerate(data_entry.items()):
        if j == 0:
            scenarios[benchmark_id_list[i]] = values
        elif j == 2:
            obj_fun[benchmark_id_list[i]] = values
        elif j == 3:
            hp_types[benchmark_id_list[i]] = values
        elif j == 1:
            dimensions[benchmark_id_list[i]] = values

dimensions_sorted = dict(sorted(dimensions.items(), key=lambda item: max(item[1]), reverse=True))
for i, (key, values) in enumerate(dimensions_sorted.items()):
    axs[1].hist(values, bins=np.arange(min(values), max(values) + 1, 3), color=colors[key])

scenarios_sorted = dict(sorted(scenarios.items(), key=lambda item: max(item[1].values()), reverse=True))
for i, (key, values) in enumerate(scenarios_sorted.items()):
    x = np.arange(len(values.keys())) + i * shift
    axs[0].bar(x, values.values(), color=colors[key], width=bar_width)
    axs[0].set_xticks(x)
    axs[0].set_xticklabels(values.keys())

obj_fun_sorted = dict(sorted(obj_fun.items(), key=lambda item: max(item[1].values()), reverse=True))
for i, (key, values) in enumerate(obj_fun_sorted.items()):
    x = np.arange(len(values.keys())) + i * shift
    axs[2].bar(x, values.values(), color=colors[key], width=bar_width)
    axs[2].set_xticks(x)
    axs[2].set_xticklabels(values.keys())

hp_types_sorted = dict(sorted(hp_types.items(), key=lambda item: max(item[1].values()), reverse=True))
for i, (key, values) in enumerate(hp_types_sorted.items()):
    x = np.arange(len(values.keys())) + i * shift
    axs[3].bar(x, values.values(), color=colors[key], width=bar_width)  # , alpha=0.5)
    axs[3].set_xticks(x)
    axs[3].set_xticklabels(values.keys())

# Add a legend to each subplot
for i, ax in enumerate(axs):
    ax.tick_params(axis="both", which="major", labelsize=22)
    ax.set_yscale("log")
    ax.set_title(list(data_entry.keys())[i], fontsize=26)

legend_handles = [mpatches.Patch(color=color, label=name) for name, color in colors.items()]

ax.legend(handles=legend_handles, fontsize=20, bbox_to_anchor=(1.05, 1), loc="upper left")

plt.tight_layout()
savefig(fig, "benchmark_footprint")
plt.show()

## Color Palette

In [ ]:
import matplotlib.pyplot as plt

# Define a colorblind-friendly color palette with 13 colors
colorblind_palette = [
    "#E69F00",
    "#56B4E9",
    "#009E73",
    "#F0E442",
    "#0072B2",
    "#D55E00",
    "#CC79A7",
    "#E69F00",
    "#56B4E9",
    "#009E73",
    "#F0E442",
    "#0072B2",
    "#D55E00",
]
colorblind_palette = ["#51A3A3", "#75485E", "#CB904D", "#DFCC74", "#C3E991"]
colorblind_palette = ["#FDC5F5", "#F7AEF8", "#B388EB", "#8093F1", "#72DDF7"]
colorblind_palette = ["#003844", "#006C67", "#F194B4", "#FFB100", "#FFEBC6"]
colorblind_palette = ["#88CCEE", "#44AA99", "#117733", "#999933", "#DDCC77", "#CC6677", "#882255", "#AA4499", "#DDDDDD"]


# Create a sample plot to show the color palette
fig, ax = plt.subplots(figsize=(10, 1), dpi=100)

for i, color in enumerate(colorblind_palette):
    ax.add_patch(plt.Rectangle((i, 0), 1, 1, color=color))

ax.set_xlim(0, len(colorblind_palette))
ax.set_ylim(0, 1)
ax.set_xticks([])
ax.set_yticks([])

plt.title("Colorblind-Friendly Palette with 13 Colors")
plt.show()